NLP Model for Sentiment Classification


Imports libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import string
import nltk
import spacy
import os
import sys

from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import en_core_web_md
text_to_nlp = en_core_web_md.load()

Imports data

In [9]:
import gdown
gdown.download("https://drive.google.com/uc?id=1u0tnEF2Q1a7H_gUEH-ZB3ATx02w8dF4p","yelp_data.csv",quiet = True)
data_file = "yelp_data.csv"

Explore Data

In [16]:
yelp_full_data = pd.read_csv(data_file)
print(yelp_full_data.head())

print("------------------------------------------------------------------------------------------------------------------------")
#Only need stars and text data
want_columns = ["stars","text"]
yelp_data = yelp_full_data[want_columns]
print(yelp_data.head())

print("------------------------------------------------------------------------------------------------------------------------")

#Look at data for different stars
star_num = 5
for s in yelp_data[yelp_data["stars"] == star_num]["text"].head(20).values:
    print(s)

              business_id  stars  \
0  9yKzy9PApeiPPOUJEtnvkg      5   
1  ZRJwVLyzEJq1VAihDhYiow      5   
2  _1QQZuf4zZOyFCvXc0o6Vg      5   
3  6ozycU1RpktNG2-1BroVtw      5   
4  zp713qNhx8d9KCJJnrw1xA      5   

                                                text                 user_id  \
0  My wife took me here on my birthday for breakf...  rLtl8ZkDX5vH5nAx9C3q5Q   
1  I have no idea why some people give bad review...  0a2KyEL0d3Yb1V6aivbIuQ   
2  Rosie, Dakota, and I LOVE Chaparral Dog Park!!...  uZetl9T0NcROGOyFfughhg   
3  General Manager Scott Petello is a good egg!!!...  vYmM4KTsC8ZfQBg-j5MWkw   
4  Drop what you're doing and drive here. After I...  wFweIWhv2fREZV_dYkz_1g   

   cool  useful  funny  
0     2       5      0  
1     0       0      0  
2     1       2      0  
3     0       0      0  
4     7       7      4  
------------------------------------------------------------------------------------------------------------------------
   stars                       